Taller Aplicado de Pivot Tables, Tablas de Contingencia y Melts

In [1]:
!wget https://github.com/javierherrera1996/lecture_analytics/raw/refs/heads/main/time_series.zip
!unzip time_series.zip
!cp -r time_series/* ./

--2024-10-17 20:23:18--  https://github.com/javierherrera1996/lecture_analytics/raw/refs/heads/main/time_series.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/refs/heads/main/time_series.zip [following]
--2024-10-17 20:23:18--  https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/refs/heads/main/time_series.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15602616 (15M) [application/zip]
Saving to: ‘time_series.zip’

time_series.zip     100%[===================>]  14.88M  83.7MB/s    in 0.2s    

2024-10-17 20:23:19 (83.7 MB/s) - ‘time

Contexto del Taller
Tienes datos históricos de precios y volumen de transacciones de acciones de una empresa (AAPL). El objetivo es utilizar herramientas como pivot tables, tablas de contingencia y melts en Python para realizar un análisis exploratorio que pueda influir en decisiones de inversión o identificar tendencias.

Preguntas de Pivot Tables (Enfocadas a Negocio)
Calcula el volumen promedio de transacciones por mes para la acción AAPL.

Contexto de negocio: El volumen mensual te ayuda a entender la liquidez de la acción y detectar patrones de alta actividad.


In [2]:
import pandas as pd
data = pd.read_csv('prices.csv')
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


Calcula el precio promedio de apertura y cierre por día de la semana.

Contexto de negocio: Esto puede ayudar a identificar si hay días de la semana en los que la acción tiende a abrir o cerrar con precios más altos.

In [5]:

data['date'] =pd.to_datetime(data['date'], format="%Y-%m-%d", errors="coerce")
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month


In [4]:
import pandas as pd

data['day_of_week'] = data['date'].dt.dayofweek

average_prices_by_day = pd.pivot_table(data, values=['open', 'close'], index='day_of_week', aggfunc='mean')

print(average_prices_by_day)

                 close       open
day_of_week                      
0.0          42.074497  42.023794
1.0          42.563750  42.520841
2.0          42.536023  42.547250
3.0          42.497981  42.438863
4.0          42.531558  42.542581


Calcula el volumen total de transacciones por día de la semana y compara la actividad entre días.

Contexto de negocio: Saber qué días tienen mayor actividad puede ayudarte a decidir cuándo realizar compras o ventas estratégicas.

In [8]:
data['date'] = pd.to_datetime(data['date'])

data['day_of_week'] = data['date'].dt.dayofweek

volumen_por_dia = data.groupby('day_of_week')['volume'].sum().idxmax()
print(f"El día de la semana con el mayor volumen de transacciones es: {volumen_por_dia}")

El día de la semana con el mayor volumen de transacciones es: 4.0


Calcula los precios máximos y mínimos de apertura por mes.

Contexto de negocio: Esto te permitirá evaluar la volatilidad mensual de la acción.

In [9]:
data.groupby(['year', 'month'])['open'].agg(['max', 'min'])

max        min
year   month                       
2010.0 1.0     17.100000  15.380000
       2.0     16.799999  15.380000
       3.0     18.799999  16.760000
       4.0     22.100000  18.620001
       5.0     23.330000  18.410000
...                  ...        ...
2016.0 8.0    124.180000  43.759998
       9.0    129.169998  43.490002
       10.0   132.449997  41.939999
       11.0   126.519997  40.009998
       12.0   126.449997  41.599998

[84 rows x 2 columns]

Calcula el volumen promedio por mes y acción (suponiendo que tienes datos de múltiples acciones).

Contexto de negocio: Esta información te ayuda a comparar la liquidez de diferentes acciones a lo largo del tiempo.


In [10]:
data.groupby(['year', 'month', 'symbol'])['volume'].mean()

year    month  symbol
2010.0  1.0    AIV       2.435384e+06
        2.0    AIV       2.366379e+06
        3.0    AIV       1.896691e+06
        4.0    AIV       2.411843e+06
        5.0    AIV       3.047175e+06
                             ...     
2016.0  11.0   FTV       1.495057e+06
               WLTW      8.967857e+05
        12.0   AIV       1.305529e+06
               FTV       1.588829e+06
               WLTW      7.636000e+05
Name: volume, Length: 102, dtype: float64

Calcula el precio de cierre promedio de la acción AAPL por mes, y agrega una columna con el total de transacciones en cada mes.

Contexto de negocio: Esto te permite comparar el comportamiento de los precios con el volumen de transacciones, para entender si hay algún patrón de venta o compra en función del precio.

In [11]:
data_AAPL=data[data["symbol"]=="AAPL"]
AAPL_close= pd.pivot_table(data_AAPL, values=["close","volume"], index="month", aggfunc={"close": "mean", "volume": "sum"})
AAPL_close.columns= ["Valor promedio de cierre", "Volumen de transacciones total"]
AAPL_close

ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements

Preguntas de Tablas de Contingencia (Aplicadas a Negocios)
Crea una tabla de contingencia para contar cuántos días el precio de apertura fue mayor o igual a $155 y cuántos días fue menor a $155.

Contexto de negocio: Esto te ayuda a entender cómo se distribuyen los precios de apertura alrededor de un umbral crítico de $155.

In [13]:
data['open_above_155'] = data['open'] >= 155
pd.pivot_table(data, values='open', index='open_above_155', aggfunc='count')

,open
open_above_155,
False,800961
True,50303


Genera una tabla de contingencia para contar cuántos días el volumen de transacciones fue mayor o igual a 75 millones y cuántos días fue menor.

Contexto de negocio: Identificar días de alto volumen de transacciones es clave para entender cuándo hubo más interés en la compra/venta de la acción.

In [14]:
data['high_volume'] = data['volume'] >= 75000000
pd.crosstab(index=data['high_volume'], columns='Count')

col_0,Count
high_volume,
False,847171
True,4093


Crea una tabla de contingencia que muestre cuántos días el precio de cierre fue mayor o igual al precio de apertura, y cuántos días fue menor.

Contexto de negocio: Saber en cuántos días la acción cerró por encima o por debajo del precio de apertura puede darte una idea general del rendimiento de la acción.

In [15]:
import numpy as np
data['close_above_open'] = np.where(data['close'] >= data['open'], 'Mayor o igual a open', 'Menor a open')
pd.crosstab(index=data['close_above_open'], columns='Count')


col_0,Count
close_above_open,
Mayor o igual a open,444764
Menor a open,406500


Añade márgenes (totales) a la tabla de contingencia anterior para conocer el total de días con subidas y bajadas de precio.

Contexto de negocio: Analizar cuántos días en general la acción tuvo subidas o bajadas te da un panorama más amplio del comportamiento de la acción.

In [16]:
data['close_open_relation'] = np.where(data['close'] > data['open'], 'Mayor a open', np.where(data['close'] == data['open'], 'Igual a open', 'Menor a open'))
pd.crosstab(index=data['close_open_relation'], columns='Count', margins=True)

col_0,Count,All
close_open_relation,,
Igual a open,7882,7882
Mayor a open,436882,436882
Menor a open,406500,406500
All,851264,851264


Crea una tabla de contingencia que compare cuántos días el precio de apertura fue mayor o igual a $155 y al mismo tiempo el precio de cierre ajustado fue mayor o menor a $155.

Contexto de negocio: Esto permite identificar patrones donde el precio de apertura y el precio de cierre ajustado cruzan umbrales críticos, lo que es útil para ajustes en las estrategias de compra/venta.

In [17]:
data['open_above_155'] = data['open'] >= 155
data['adj_close_above_155'] = data['close'] >= 155
pd.crosstab(data['open_above_155'], data['adj_close_above_155'], rownames=['Open >= 155'], colnames=['Adj Close >= 155'])

Adj Close >= 155,False,True
Open >= 155,,
False,800422,539
True,489,49814


Preguntas de Melts (Aplicadas a Negocios)
Transforma las columnas de precios (Open, Close, High, Low) en una única columna llamada Precio, manteniendo la fecha (Date) y el tipo de acción (Stock).

Contexto de negocio: Este formato largo es ideal para realizar visualizaciones comparativas de diferentes precios de la acción a lo largo del tiempo.

In [18]:
pd.melt(data, id_vars=['date', 'symbol'], value_vars=['open', 'close', 'high', 'low'], var_name='Price_Type', value_name='Price')

,date,symbol,Price_Type,Price
0,2016-01-05,WLTW,open,123.430000
1,2016-01-06,WLTW,open,125.239998
2,2016-01-07,WLTW,open,116.379997
3,2016-01-08,WLTW,open,115.480003
4,2016-01-11,WLTW,open,117.010002
...,...,...,...,...
3405051,NaT,ZBH,low,102.849998
3405052,NaT,ZION,low,42.689999
3405053,NaT,ZTS,low,53.270000
3405054,2016-12-30,AIV,low,44.410000


Convierte todas las columnas del dataset, excepto Date y Stock, en un formato largo usando melt.

Contexto de negocio: Tener todas las variables en un formato largo es útil para generar reportes dinámicos que comparen diferentes tipos de datos (precios, volumen, etc.).

In [19]:
pd.melt(data, id_vars=['date', 'symbol'], var_name='variable', value_name='value')

,date,symbol,variable,value
0,2016-01-05,WLTW,open,123.43
1,2016-01-06,WLTW,open,125.239998
2,2016-01-07,WLTW,open,116.379997
3,2016-01-08,WLTW,open,115.480003
4,2016-01-11,WLTW,open,117.010002
...,...,...,...,...
11917691,NaT,ZBH,adj_close_above_155,False
11917692,NaT,ZION,adj_close_above_155,False
11917693,NaT,ZTS,adj_close_above_155,False
11917694,2016-12-30,AIV,adj_close_above_155,False


Usa melt para convertir los precios (Open, Close, High, Low) y el volumen (Volume) en una sola columna de valor. Luego, agrupa los datos por tipo de precio y acción para calcular el promedio de cada tipo.

Contexto de negocio: Este análisis permite comparar el comportamiento de diferentes precios y el volumen en un solo análisis, lo cual es útil para entender tendencias generales de la acción.

In [20]:
melted_data = pd.melt(data, id_vars=['date', 'symbol'], value_vars=['open', 'close', 'high', 'low', 'volume'], var_name='Price_Type', value_name='Value')
melted_data.groupby(['Price_Type', 'symbol'])['Value'].mean()

Price_Type  symbol
close       A         4.253944e+01
            AAL       2.387377e+01
            AAP       1.037696e+02
            AAPL      3.129271e+02
            ABBV      5.547364e+01
                          ...     
volume      YHOO      1.996907e+07
            YUM       5.163684e+06
            ZBH       1.509946e+06
            ZION      3.227187e+06
            ZTS       4.020115e+06
Name: Value, Length: 2505, dtype: float64

Realiza una transformación melt para convertir los precios en una columna y agrúpalos por día para calcular el precio promedio diario de la acción.

Contexto de negocio: Esto proporciona una visión agregada del comportamiento promedio de los precios diarios de la acción.

In [25]:
melted_data = pd.melt(data, id_vars=['date'], value_vars=['open', 'close', 'high', 'low'],var_name='Price_Type', value_name='Price')

average_daily_price = melted_data.groupby('date')['Price'].mean()
